# Input data and import modules
## Here we input the directory for all the results of the Fiji plugin Comdet v0.5.5 and the _seg.py files, See the MAIN CODE below for more details

In [13]:

import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from cellpose import plot, utils
import statistics as s 
import sys, os
import pandas as pd
import math
import copy
from matplotlib.ticker import PercentFormatter

#-------------------INPUT THE DIRECTORIES HERE---------:

#Directory for the output "Results" from the "particle_detection" script for FIJI

directory_res = "/Users/ceciliaacosta/Downloads/data ceci/1rst dt/results/"

#Directory for the output of CELLPOSE segmentation masks in which you put _seg.py files only

directory_seg="/Users/ceciliaacosta/Downloads/data ceci/1rst dt/seg/"

#This iterate over files in directory_res and put it in a list 

files=[]
for filename in os.listdir(directory_res):
    f = os.path.join(directory_res, filename)
    if not filename.endswith(".DS_Store"):
        files.append(f)

files.sort()

### This deletes all the segmentation files from the channel that does not have good segmentation 

In [25]:
folder_path = "/Users/ceciliaacosta/Downloads/data ceci/1rst dt/seg/"  # Replace with the path to your folder
files = os.listdir(folder_path)
files.sort()
for i, file in enumerate(files):
    if i % 2 == 0: #Keep the even numbered files (removes 1, 3, 5, etc...)

        os.remove(os.path.join(folder_path, file))

directory_seg="/Users/ceciliaacosta/Downloads/data ceci/1rst dt/seg/"


# Functions 

## Function: Point polygon algorithm
- INPUT: the outlines of each mask as vertices and the pixel coordinate (X,Y) at the center of the particle detection.
- OUTPUT: True if inside, False if outside

## Function to get the cell lenght
- INPUT: An array/list of coordinates that define the outline of a cell
- OUTPUT: Two variables: - The length of the cell, - The coordinate at the center of the cell 

In [2]:
def point_in_poly(x, y, poly):
    # Check if a point (x,y) is inside a polygon defined by a set of vertices 
  
    n = len(poly)
    inside = False

    p1x, p1y = poly[0]
    
    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

def cell_lenght(outline):
    #---initialize the min, max values of the coordinates of the outline of one cell
    maxx= float('-inf')
    maxy= float('-inf')
    minx= float('inf')
    miny= float('inf')

#--This for loop finds the max and min value for x and y coordinates: maxx= max (x) pixel coord, minx= min (x) pixel coord, maxy= max (y) pixel coord, miny= "" ""
    for i in range (len(outline)):

        x=outline[i][0]
        y=outline[i][1]
        
        if x>maxx:
            maxx=x
        if y>maxy:
            maxy=y
        if y<miny:
            miny=y
        if x<minx:
            minx=x

    lenght=math.sqrt((maxx-minx)**2+(maxy-miny)**2) # Find the lenght using the Pythagoras formula
    center_coordinate= [minx+(maxx-minx)/2, miny+(maxy-miny)/2]    # Find the approximate center of the cell

    return lenght, center_coordinate


## Function to create a Table to be able to do data analysis
- INPUT: fiji file in a readable form, the outlines of the cell from the _seg.npy segmentation file from cellpose, x: the size of the pixel
- It outputs a table where we have the relative distance of a particle to its corresponding cell center, and many other datas. It creates a table that will be saved and outputed in the MAIN CODE and also outputs the counts per cell in an image that will be used to create the histogram of counts per cell for a channel. 

In [3]:
def table_creation(data, outlines, size, min): 

#-----------Initialization of variables -----------
    counts= np.empty(0, dtype=int)   #The number of particles per cell 
    table= np.zeros((1000, 10)) #the final table with 10 columns and 1000 rows (1000 is arbitrary to not cause error)
    count= 0
    one= []    #one ,two , ... are lists that will contain the counts for each different classes of cell lenght chosen
    two= []
    three= []
    four= []
    rel_dist= []
    z= 0       #this var 
    coloc= {}
    cellnum=1


    #-----------create tables
    for i in range (len(outlines)):
        poly=outlines[i]
        lenght, center= cell_lenght(poly)
        if lenght>min :

            count=0

            for j in range ( len(data)):
                #MAKE SURE TO FIRST OPEN THE RESULTS DATA TO SEE HOW THE COLUMNS WERE SAVED (sometimes FIJI saves the results with one extra empty column at the beginning)
                x=data[j][1]
                y=data[j][2]
                coord=[x,y]
                
                if point_in_poly(x,y,poly): #if particle inside the cell segmentation
        
                    distance= math.sqrt((y-center[1])**2+(x-center[0])**2)
                    rel_dist.append(distance/(lenght/2))

                    count+=1
            
                    table[z][0]=i
                    table[z][1]=count 
                    table[z][3]=lenght
                    table[z][4]=x
                    table[z][5]=y
                    table[z][6]=data[j][11] #Integrated Intensity of the particle detected
                    table[z][7]=center[0]
                    table[z][8]=center[1]
                    table[z][9]=distance

                    if cellnum not in coloc:

                        coloc[cellnum]=[]
                        coloc[cellnum].append(coord)
                    
                    else:

                        coloc[cellnum].append(coord)
                
                    z+=1

            counts=np.append(counts,count)
            #print(cellnum, count)
            if (count==0):
                table[z][0]=i
                table[z][3]=lenght
                table[z][7]=center[0]
                table[z][8]=center[1] #the rest of the columns will be 0     
                coloc[cellnum]=[]

                z+=1
                            
            else:
                y=z
                for w in range (0,count):
                    table[y-1][2]=count
                    y-=1
            
            cellnum+=1
            
            #-----------change variable "function" to True or False depending if you want histograms for each diff cell size------

            function=True

            if (function):

                micron= size*lenght

                if (micron>=2000 and micron<=2800):
                    one.append(count)
                elif (micron> 2800 and micron<=3600):
                    two.append(count)
                elif (micron> 3600 and micron<=4400):
                    three.append(count)
                elif (micron> 4400):
                    four.append(count)
                else:
                    continue 
        else:
            continue

    average=s.mean(counts)

    return table, counts, average, one, two, three, four, coloc, rel_dist, cellnum

## Functions for histograms

In [8]:
def counts_histogram(total_1, total_2):

    fig=plt.figure()
    bins1=np.arange(0,int(max(np.max(total_1), np.max(total_2))), 1)
    ax = fig.add_subplot(111)
    ax.hist(total_1, weights=np.ones(len(total_1)) / len(total_1),  alpha=0.6,label="channel 1,  n="+str(sum(total_1)), ec="black", rwidth = 0.5,  bins=bins1-0.5, facecolor='yellow')
    ax.hist(total_2, weights=np.ones(len(total_2)) / len(total_2), alpha=0.75, ec="black", label="channel 2,  n="+str(sum(total_2)), rwidth = 0.5,  bins=bins1-0.25, facecolor='red')
    plt.xticks(bins1)
    ax.set_title("Particles per cell for each channel")
    # set the x-axis label
    ax.set_xlabel('Particles per cell')
    # set the y-axis label
    ax.set_ylabel('Percentage for each channel')
    
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.legend(loc="upper right")
    plt.savefig('Counts.png')

def size_histogram(one, two, three, four, chan, total):

    fig=plt.figure()
    bins1=np.arange(6)
    ax = fig.add_subplot(111)
    ax.hist(one, weights=np.ones(len(one)) / len(total),  alpha=0.6, label='2 ≤size< 2.8'+', n=' +str(len(one)), ec="black", rwidth = 0.2,  bins=bins1-0.8, facecolor='yellow')
    ax.hist(two, weights=np.ones(len(two)) / len(total), alpha=0.75, label='2.8 ≤size< 3.6'+', n=' +str(len(two)), ec="black", rwidth = 0.2,  bins=bins1-0.60, facecolor='red')
    ax.hist(three, weights=np.ones(len(three)) / len(total),  alpha=0.6, label='3.6 ≤size< 4.4'+', n=' +str(len(three)), ec="black", rwidth = 0.2,  bins=bins1-0.40, facecolor='blue')
    ax.hist(four, weights=np.ones(len(four)) / len(total), alpha=0.75, label='4.4 ≤size'+', n='+str(len(four)), ec="black", rwidth = 0.2,  bins=bins1-0.20, facecolor='green')
    ax.set_title("Particle per cell for different cell sizes for channel:"+str(chan))
    # set the x-axis label
    ax.set_xlabel('Particles per cell')
    # set the y-axis label
    ax.set_ylabel('Proportion of particles')
    plt.legend(fontsize='small')
    plt.xticks(bins1)
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.legend(loc="upper right")
    plt.savefig(str(chan)+'cell_size.png')


def coloc_histogram(data, tot_channel, title, pix, channel):
    total=sum(tot_channel)
    fig, ax = plt.subplots()
    print("There is "+str(total)+" particles for the channel: " + str(channel))
    x_axis=[]
    for size in data.keys():
        #print(size)
        x_axis.append(size*pix)

# plot the data as a bar chart
    #bars=ax.bar(data.keys(), [i/total for i in data.values()], label='n='+ str(total),  ec="black",  width = 0.5)
    
    bars=ax.bar(x_axis, [j/total for j in data.values()], label='n='+ str(total)+ 'in chan 1',  ec="black", width = 10)

    # set the y-axis limit
    ax.set_ylim(0, 1)
    # set the x-axis label
    ax.set_xlabel('Max distance in microns')
    # set the y-axis label
    ax.set_ylabel('Colocalization rate')
    # set the title
    ax.set_title(str(title))

    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')

    plt.xticks(x_axis)
    
    plt.legend(loc="upper right")
    # show the plot
    plt.savefig('coloc.png')
    plt.show()

def distance_histogram(data, total, channel):
    tot=sum(total)
    fig, ax = plt.subplots()
    bin_width= 1/40
    bin_edges= np.arange(0, 0.51, bin_width)
    hist, edges= np.histogram(data/2, bins=bin_edges)

# plot the data as a bar chart
    plt.bar(edges[:-1], hist, width=bin_width, align='edge', label='n='+ str(sum(total)), ec="black")

    # set the y-axis limit
    ax.set_xlim(0, 0.5)
    # set the x-axis label
    ax.set_xlabel('Absolute relative distance to the center')
    # set the y-axis label
    ax.set_ylabel('Quantity of particles')
    # set the title
    ax.set_title("Histogram of the relative distance of particles\n from the cell center for Channel: "+str(channel))

    # for bar in bars:
    #     height = bar.get_height()
    #     ax.text(bar.get_x() + bar.get_width() / 2, height, f'{height:.2f}', ha='center', va='bottom')

    #plt.xticks(x_axis)
    #plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    plt.legend(loc="upper right")
    # show the plot
    plt.savefig(str(channel)+'distance.png')
    plt.show()

## Functions for colocalization within segmentations

In [7]:
def allowed_dist (cell1, cell2, maxcol_dist):
    dist= math.sqrt((cell1[0]-cell2[0])**2+(cell1[1]-cell2[1])**2)
    if (dist>maxcol_dist):

        return False, dist
    else: 
        return True, dist


def colocalization(Input_chan, other_chan, n, pixel):
    result=0
    boo=False
    candidate=float('inf')
    inp=0
    out=0
    for key1 in Input_chan.keys(): #iterates through all the cells of the image 

        if len(Input_chan[key1])==0: #if the cell chosen chanel has no particle go to next cell
            continue

        elif len(other_chan[key1])==0: #if the cell other chanel has no particle return result=0 
            continue         
        
        else: 
            inp+=len(Input_chan[key1])
            for j in range(0, len(Input_chan[key1])):
                smallest_dist=float('inf')
                
                for y in range (0, len(other_chan[key1])): 
                    allowed, dist= allowed_dist(Input_chan[key1][j], other_chan[key1][y], pixel )

                    if allowed and dist<=smallest_dist:
                        smallest_dist=dist
                        candidate=y
                        boo=True
                    else:
                        continue     
                if boo:
                    del other_chan[key1][candidate]
                    result+=1
                boo=False
                
    return result


# MAIN CODE
## Variable definitions:

- **total_1** and **total_2**: Are lists for each channel where we have the COUNT of particule for each cells for ALL images. We use these variables to create the histogram
- **fiji_2** and **fiji_2** : Input variable, represents the Results output from the ComDet v.0.5.5 Detect particles Fiji Plugin. It is important to retrieve the Integrated Intensity of the particle and the pixel coordinates.
    - When the  is a .txt file, use the delimiter='\t'. When it is a .csv file use delimiter=','

- **outlines** : Important input variable for this script: we take this variable from the _seg.py files created by CELLPOSE output after segmentation. We can use the segmentation of the best color channel instead of segmentating twice the same image (but diff channel).

## Specifications: 

- In this case, each segmentation **_seg.py** file chosen from img of a single color channel will cover Particle Detection Results for BOTH channels. This is to avoid bias of segmentation that could be different for each channel.

In [22]:
pixel= int(input("What is the pixel size in nanometer? Input a number:"))
channel=int(input("which chanel do you want for colocalizaton? Input either 1 or 2:"))
too_small=float(input("What is the minimum cell lenght from the segmentation data allowed? Please enter an number:"))

names=['cell_id', 'part_id','total_count','cell_lenght', 'Part_X_(px)', 'Part_Y_(px)', 'IntegatedIntensity', 'Cellcenter_X(px)', 'Cellcenter_Y(px)', 'Distance']

#------ #total particles for each channel --------------------------------------------------------------------------------------------------------------------------------------------
total_1=np.empty(0) 
total_2=np.empty(0)

#------ Variables to make the histogram for particles per cell size------------------------------------------------------------------------------------------------------------------
tot_1=np.empty(0) #for channel 1 divided in cell size
tot_2=np.empty(0)
tot_3=np.empty(0) 
tot_4=np.empty(0)
tot_1_2=np.empty(0) #for channel 2 divided in cell
tot_2_2=np.empty(0)
tot_3_2=np.empty(0)
tot_4_2=np.empty(0) 

#------Variable to make the histogram of colocalization rate per distance size in pixels----------------------------------------------------------------------------------------------
new_dict = {0.5:0, 1.0:0, 1.5:0, 2:0, 2.5:0, 3:0, 3.5:0, 4:0 }
dist1=np.empty(0)
dist2=np.empty(0)

#---------Variables for the intensity-------------------------------------------
intensity=np.empty(0)



i=0
for filename in sorted(os.listdir(directory_seg)):
 
    if not filename.endswith(".DS_Store"):
        print(filename)
    
        seg_file = os.path.join(directory_seg, filename)
        seg_masks=np.load(seg_file, allow_pickle=True).item()
        masks=seg_masks['masks']
        outlines= utils.outlines_list(seg_masks['masks'])
        n=len(outlines)


        fiji_1= np.genfromtxt(files[2*i], delimiter='\t', skip_header=1)   # input channel 1
        fiji_2= np.genfromtxt(files[2*i+1], delimiter='\t', skip_header=1) # input channel 2
        i+=1

        table_1, counts_1, average_1, one, two, three, four, coloc, distance, cellnum1=table_creation(fiji_1, outlines,pixel, too_small)
 
        table_2, counts_2, average_2, one2, two2, three2, four2, coloc2, distance2, cellnum2 =table_creation(fiji_2, outlines,pixel, too_small)  # channel 2

    #------- Use the returned tables to concatenate in an empty numpty array some infor-------

    #into three different empty arrays 
    # one for intensity
    # one for lenght of cell 
    # one for particle count 
    # one for distance from the center 

    # then we wiill evaluate the intensity as a function of the other three arrays in a scatter plot and maybe in a violin plot for the particle count 

    #------------------# of particle per cell histogram for both channel -------------

        total_1 =np.concatenate((total_1, counts_1), axis=None) 
        total_2 =np.concatenate((total_2, counts_2), axis=None)

    #--------------------PARTICLE PER CELL SIZE HISTOGRAM-----------------------------
        tot_1=np.concatenate((tot_1, one), axis=None) 
        tot_2=np.concatenate((tot_2, two), axis=None) 
        tot_3=np.concatenate((tot_3, three), axis=None) 
        tot_4=np.concatenate((tot_4, four), axis=None) 

        tot_1_2=np.concatenate((tot_1_2, one2), axis=None) 
        tot_2_2=np.concatenate((tot_2_2, two2), axis=None) 
        tot_3_2=np.concatenate((tot_3_2, three2), axis=None) 
        tot_4_2=np.concatenate((tot_4_2, four2), axis=None) 

        dist1=np.concatenate((dist1,distance),  axis=None)
        dist2=np.concatenate((dist2,distance2),  axis=None)

    #-----------------Saving results of tables created here in CSV file ---------------

        df1 = pd.DataFrame(table_1, columns=names)
        df1.to_csv('First_channel'+str(i)+'.csv', index=True, header=True, sep=',')
        df2 = pd.DataFrame(table_2, columns=names)
        df2.to_csv('Second_channel'+str(i)+'.csv', index=True, header=True, sep=',')
    #----------------- COLOCALIZATION

        for key in new_dict.keys():

            inco=copy.deepcopy(coloc)
            othco=copy.deepcopy(coloc2)
            #print(othco)
            rate=colocalization(inco, othco, cellnum1, key)
            new_dict[key]+=rate 


#--------------Plot the histrogram of counts per cell
counts_histogram (total_1, total_2)

#--------------Plot histogram for each size of cell
size_histogram (tot_1, tot_2, tot_3, tot_4, 1, total_1)
size_histogram (tot_1_2, tot_2_2, tot_3_2, tot_4_2, 2, total_2)
    
# #-------------Plot histogram for colocalization
if channel==1:
    coloc_histogram(new_dict,total_1, "Channel 1 on channel 2 ", pixel, channel)
else:
    coloc_histogram(new_dict,total_2, "Channel 2 on channel 1", pixel, channel)

distance_histogram(dist1, total_1, 1)
distance_histogram(dist2, total_2, 2)


SyntaxError: invalid syntax (3286603514.py, line 64)